<a href="https://colab.research.google.com/github/JoaoJanini/seq2seq/blob/fix%2Fswitch_to_seq_to_seq_trainer/notebooks/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import repo

In [2]:
%rm -rf /content/seq2seq
import getpass
!git clone --branch fix/switch_to_seq_to_seq_trainer https://{getpass.getpass()}@github.com/JoaoJanini/seq2seq

··········
Cloning into 'seq2seq'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 289 (delta 154), reused 220 (delta 92), pack-reused 0
Receiving objects: 100% (289/289), 1.38 MiB | 4.45 MiB/s, done.
Resolving deltas: 100% (154/154), done.


In [3]:
!sleep 5

In [4]:
%cd seq2seq

/content/seq2seq


In [5]:
%mkdir data

In [6]:
!pip install -r requirements.txt 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 7.5 MB/s 
     |████████████████████████████████| 5.5 MB 82.7 MB/s 
     |████████████████████████████████| 175 kB 17.3 MB/s 
     |████████████████████████████████| 59.1 MB 276 kB/s 
     |████████████████████████████████| 451 kB 80.7 MB/s 
     |████████████████████████████████| 125 kB 86.5 MB/s 
     |████████████████████████████████| 5.3 MB 72.7 MB/s 
     |████████████████████████████████| 72 kB 1.6 MB/s 
     |████████████████████████████████| 7.6 MB 72.9 MB/s 
     |████████████████████████████████| 182 kB 73.8 MB/s 
     |████████████████████████████████| 212 kB 34.5 MB/s 
     |████████████████████████████████| 115 kB 69.5 MB/s 
     |████████████████████████████████| 127 kB 70.1 MB/s 
     |████████████████████████████████| 8.8 MB 58.5 MB/s 
     |████████████████████████████████| 468 kB 68.8 MB/s 
  Attempting uninstall: urllib3
  

# Sequence to Sequence

In [7]:
import sys
path = "./seq2seq"
if path not in sys.path:
  sys.path.append(path)
else:
  print(path + "already in path")

In [8]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 './seq2seq']

In [ ]:
from transformers import TrainingArguments, Trainer, logging, Seq2SeqTrainingArguments, Seq2SeqTrainer
from hf_sequence_to_sequence.model import FaciesForConditionalGeneration
from hf_sequence_to_sequence.configuration import FaciesConfig
import torchmetrics
import math
import time
from torch import nn, optim
from torch.optim import Adam
import torch
from torch.utils.data import DataLoader
from dataset.dataset import WellsDataset
from torch.utils.data import random_split
from typing import List
from datetime import datetime
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from utils import collate_fn
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.search.hyperopt import HyperOptSearch
from ray import tune
# define function to compute metrics
import numpy as np


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 64
EVAL_BATCH_SIZE = 640
SEQUENCE_LEN = 15
TRAINING_RATIO = 0.90
WIRELINE_LOGS_HEADER = ["GR", "NPHI", "RSHA", "DTC", "RHOB", "SP"]
LABEL_COLUMN_HEADER = ["FORCE_2020_LITHOFACIES_LITHOLOGY"]
model_directory = f"/content/drive/MyDrive/Coding/seq2seq/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"

train_dataset = WellsDataset(
    dataset_type="train",
    sequence_len=SEQUENCE_LEN,
    model_type="seq2seq",
    feature_columns=WIRELINE_LOGS_HEADER,
    label_columns=LABEL_COLUMN_HEADER,
)
test_dataset = WellsDataset(
    dataset_type="test",
    sequence_len=SEQUENCE_LEN,
    model_type="seq2seq",
    feature_columns=WIRELINE_LOGS_HEADER,
    label_columns=LABEL_COLUMN_HEADER,
    scaler=train_dataset.scaler,
    output_len=train_dataset.output_len,
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn
)

DATA_LEN = train_dataset.train_len
d_input = train_dataset.input_len
d_output = train_dataset.output_len
d_channel = train_dataset.channel_len
tgt_vocab_size = train_dataset.output_len + len(train_dataset.special_symbols)
TRAIN_DATA_LEN = int(DATA_LEN * TRAINING_RATIO)

train_data, validation_data = random_split(
    train_dataset, lengths=[TRAIN_DATA_LEN, DATA_LEN - TRAIN_DATA_LEN]
)


facies_config = {
    "vocab_size": tgt_vocab_size,
    "max_position_embeddings": 1024,
    "encoder_layers": 6,
    "encoder_ffn_dim": 1024,
    "encoder_attention_heads": 8,
    "decoder_layers": 6,
    "decoder_ffn_dim": 1024,
    "decoder_attention_heads": 8,
    "encoder_layerdrop": 0.0,
    "decoder_layerdrop": 0.0,
    "activation_function": "relu",
    "d_model": 1024,
    "n_input_features": d_input,
    "n_output_features": d_output,
    "sequence_len": SEQUENCE_LEN,
    "dropout": 0.2,
    "attention_dropout": 0.0,
    "activation_dropout": 0.0,
    "init_std": 0.02,
    "classifier_dropout": 0.0,
    "scale_embedding": False,
    "use_cache": False,
    "num_labels": tgt_vocab_size,
    "pad_token_id": train_dataset.PAD_IDX,
    "bos_token_id": train_dataset.PAD_IDX,
    "eos_token_id": train_dataset.PAD_IDX,
    "is_encoder_decoder": True,
    "decoder_start_token_id": train_dataset.PAD_IDX,
    "forced_eos_token_id": train_dataset.PAD_IDX,
    "return_dict": False,
}
facies_transformer_config = FaciesConfig(**facies_config)
facies_transformer_config.save_pretrained(
    f"{model_directory}/facies-transformer-config"
)
facies_transformer_config = FaciesConfig.from_pretrained(
    f"{model_directory}/facies-transformer-config"
)


# function to collate data samples into batch tesors
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([16, 640, 1280]),
        "weight_decay": tune.uniform(0.0, 0.3),
        "num_train_epochs": tune.choice([5, 10, 15]),
        "dropout": tune.choice([0.3, 0.1, 0.2, 0.0, 0.4]),
    }

def compute_metrics_fn(eval_preds):
    metrics = dict()
    accuracy_metric = load_metric("accuracy")
    preds = eval_preds.predictions[:, 1:-1]
    preds = preds.flatten()
    labels = eval_preds.label_ids[:,:-2]
    labels = labels.flatten()
    preds = preds[labels != 0]
    labels = labels[labels != 0]

    metrics.update(accuracy_metric.compute(predictions=preds, references=labels))

    return metrics


def model_init(trial):
    model_config = facies_transformer_config
    if trial is not None:
        model_config.update({"dropout": trial["dropout"]})
    print("model_init() called. updated config is")
    print(model_config)

    return FaciesForConditionalGeneration(model_config)

    return FaciesForConditionalGeneration(facies_transformer_config)


training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_directory}/facies-transformer",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    evaluation_strategy="epoch",
    num_train_epochs=10,
    generation_max_length=SEQUENCE_LEN + 2,
    generation_num_beams=4,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=None,
    train_dataset=train_data,
    data_collator=collate_fn,
    eval_dataset=validation_data,
    args=training_args,
    model_init = model_init,
    compute_metrics=compute_metrics_fn
)

best_model = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    n_trials=10,
    search_alg=HyperOptSearch(metric="objective", mode="max"),
    hp_space=ray_hp_space,
    local_dir=f"{model_directory}/ray_results",
    resources_per_trial={"gpu": 1, "cpu":1}
)


